In [1]:
import pandas as pd
import altair as alt

In [2]:
all_teams_2019_2024_df = pd.read_csv("all_teams_playoff_vs_season_stats_2019_2024.csv")
all_players_2019_2024_df = pd.read_csv("all_players_playoff_vs_season_stats_2019_2024.csv")

In [7]:
hover = alt.selection_point(on = 'pointerover',nearest = False, empty = 'none', clear = 'mouseout')
# click = alt.selection_point(empty = False, on = "click", name = "click")
team_select = alt.selection_point(fields=['Tm'], on='click', empty='none', name='TeamClick')

hover_and_click_condition_opacity = (alt.when(team_select).then(alt.value(1)).when(hover).then(alt.value(.75)).otherwise(alt.value(.50)))
hover_and_click_condition_size = (alt.when(team_select).then(alt.value(300)).when(hover).then(alt.value(200)).otherwise(alt.value(100)))


year_slider = alt.binding_range(
    min=2019, 
    max=2024, 
    step=1, 
    name='Select Year:'
)
year_selection = alt.selection_point(
    fields=['season_year'],
    bind=year_slider,
    value=2024  # Default selected year
)

standings = alt.Chart(all_teams_2019_2024_df).mark_image(
    width=40,
    height=40
).encode(
    y=alt.Y('Rk_playoff:Q', scale=alt.Scale(domain=[16,1]), title='', axis=alt.Axis(grid=False)),
    x=alt.X('x:Q', axis=alt.Axis(grid=False, domain=False, domainOpacity=0), title=''),
    url='logo_url:N',
    tooltip=['Tm', 'W_playoff', 'L_playoff', 'season_year'],
    opacity=hover_and_click_condition_opacity,
).add_params(
    hover,
    team_select,
    year_selection  # Add the year selection parameter
).transform_filter(
    year_selection  # Filter data based on the selected year
).transform_calculate(
    x='0'
).properties(
    title='Playoff Rankings',
    height=600,
    width=100
)

def create_team_stat_chart(stat):
    chart = alt.Chart(all_teams_2019_2024_df).mark_image(width=30, height=30).encode(
        alt.X(f'{stat}_in_season:Q', title="In Season", scale=alt.Scale(zero=False, padding=10)),
        alt.Y(f'{stat}_playoff:Q', title="Playoffs", scale=alt.Scale(zero=False, padding=10)),
        url='logo_url:N',
        tooltip=[alt.Tooltip('Tm:N', title="Team"),
                alt.Tooltip('Rk_playoff', title='Rank in Playoff'),
                alt.Tooltip(f'{stat}_in_season', title=f"In Season {stat}"),
                alt.Tooltip(f'{stat}_playoff', title= "Playoff stat")],
        size=hover_and_click_condition_size,
        # opacity = hover_and_click_condition_opacity
        opacity=alt.condition(team_select, alt.value(1), alt.value(0.25))
    ).add_params(
        hover, team_select, year_selection
    ).transform_filter(
        year_selection
    ).properties(
        title = f"Team Level Rating ({stat})",
        height = 300,
        width = 300
    ).interactive()

    return chart




stat_options = ['PER', 'OBPM', 'DBPM', 'BPM', 'VORP', 'WS']

# Stat dropdown parameter
stat_param = alt.param(
    name='stat',
    bind=alt.binding_select(options=stat_options, name='Select Stat:'),
    value="WS"
)

def make_stat_layer(stat_name):
    # Player points chart
    stat_chart = alt.Chart(all_players_2019_2024_df).mark_circle(size=100).encode(
        x=alt.X(f'{stat_name}_in_season:Q', title='Regular Season'),
        y=alt.Y(f'{stat_name}_playoff:Q', title='Playoffs'),
        opacity=alt.condition(team_select, alt.value(.75), alt.value(0.25)),
        size=alt.condition(team_select, alt.value(150), alt.value(100)),
        tooltip=[
            alt.Tooltip('Player'),
            alt.Tooltip('Tm_playoff', title='Playoff Team'),
            alt.Tooltip(f'{stat_name}_playoff:Q', title='Playoff Stat'),
            alt.Tooltip(f'{stat_name}_in_season:Q', title='Regular Season Stat'),
        ],
        color=alt.condition(
            team_select,
            alt.Color('Tm_playoff:N', scale=alt.Scale(scheme='tableau10')),
            alt.value('lightgray')
        )
    ).transform_calculate(
        stat=f'"{stat_name}"'
    ).transform_filter(
        'datum.stat === stat'
    ).transform_filter(
        year_selection
    )


    # Label overlay for this stat (constant dummy data)
    text_df = pd.DataFrame([{'x': 0, 'y': 1, 'stat': stat_name, 'season_year': y} for y in range(2019, 2025)])

    text_chart = alt.Chart(text_df).mark_text(
        align='left',
        baseline='top',
        dx=10,
        dy=10,
        fontSize=25,
        fontWeight='bold',
        color='black'
    ).encode(
        x=alt.value(0),
        y=alt.value(0),
        text=alt.value(stat_name),
        opacity=alt.condition('datum.stat === stat', alt.value(0.6), alt.value(0))
    ).transform_filter(
        'datum.stat === stat'
    ).transform_filter(
        year_selection
    )

    return stat_chart + text_chart


# Compose full chart with all stat layers (but only one visible at a time)
player_stats_chart = alt.layer(
    *[make_stat_layer(stat) for stat in stat_options]
).add_params(
    stat_param, hover, team_select, year_selection
).interactive()





ratings = (
    (create_team_stat_chart("NRtg") | player_stats_chart)
    & (create_team_stat_chart("ORtg") | create_team_stat_chart("DRtg"))
)
standings | ratings

alt.HConcatChart(...)